Прочтём данные из файлов с результатами A/B-тестирования:

In [ ]:
import pandas as pd

In [ ]:
orders = pd.read_csv('datasets/orders.csv', parse_dates=['date'])

In [ ]:
visitors = pd.read_csv('datasets/visitors.csv', parse_dates=['date'])

In [ ]:
orders

In [ ]:
visitors

In [ ]:
def cumulate_column(df, column):
    grouped_by_a = df[df['group'] == 'A'][column].cumsum()
    grouped_by_b = df[df['group'] == 'B'][column].cumsum()
    cumulated = pd.concat([grouped_by_a, grouped_by_b]).reset_index()
    cumulated.sort_values(by='index', inplace=True)
    return cumulated.set_index('index')

In [ ]:
visitors_aggregated = visitors.groupby(['date','group'], as_index=False).agg({'date' : 'max',
                                                                              'group' : 'max',
                                                                              'visitors' : 'sum'}, axis=1)

In [ ]:
orders_aggregated = orders.groupby(['date','group'], as_index=False).agg({'date' : 'max',
                                                                          'group' : 'max',
                                                                          'transactionId' : 'nunique',
                                                                          'visitorId' : 'nunique',
                                                                          'revenue' : 'sum'}, axis=1)

In [ ]:
visitors_aggregated

In [ ]:
orders_aggregated

In [ ]:
cumulate_column(visitors_aggregated, 'visitors')

In [ ]:
cumulate_column(orders_aggregated, 'revenue')

In [ ]:
visitors_aggregated['visitors'] = cumulate_column(visitors_aggregated, 'visitors')
orders_aggregated['transactionId'] = cumulate_column(orders_aggregated, 'transactionId')
orders_aggregated['visitorId'] = cumulate_column(orders_aggregated, 'visitorId')

In [ ]:
visitors_aggregated

In [ ]:
orders_aggregated

In [ ]:
cumulativeData = orders_aggregated.merge(visitors_aggregated, left_on=['date', 'group'], right_on=['date', 'group'])
cumulativeData.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

cumulativeData.head(11)

In [ ]:
cumulativeData = orders_aggregated.merge(visitors_aggregated)
cumulativeData.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

cumulativeData

In [ ]:
visitors[visitors['date'] == '2019-08-05']

In [ ]:
orders[orders['date'] == '2019-08-05'].count()

In [ ]:
temp = orders[orders['date'] == '2019-08-05']

In [ ]:
temp.groupby(['date'], as_index=False).agg({'date' : 'max',
                                            'group' : 'max',
                                            'transactionId' : 'nunique',
                                            'visitorId' : 'nunique',
                                            'revenue' : 'sum'}, axis=1)

In [ ]:
import matplotlib.pyplot as plt

# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе А
cumulativeRevenueA = cumulativeData[cumulativeData['group']=='A'][['date','revenue', 'orders']]

# датафрейм с кумулятивным количеством заказов и кумулятивной выручкой по дням в группе B
cumulativeRevenueB = cumulativeData[cumulativeData['group']=='B'][['date','revenue', 'orders']]

# Строим график выручки группы А
plt.plot(cumulativeRevenueA['date'], cumulativeRevenueA['revenue'], label='A')

# Строим график выручки группы B
plt.plot(cumulativeRevenueB['date'], cumulativeRevenueB['revenue'], label='B')

plt.legend()

In [ ]:
from solver.ab_reporter import ABReporter

In [ ]:
example = ABReporter('datasets/visitors.csv', 'datasets/orders.csv')

In [ ]:
example.grouped_summary()

Аналогично получим агрегированные кумулятивные по дням данные о посетителях интернет-магазина:


In [ ]:
df = visitors.groupby(['date','group'], as_index=False).agg({'date' : 'max',
                                                        'group' : 'max',
                                                        'visitors' : 'sum'}, axis=1).sort_values(by=['date','group'])

In [ ]:
# создаем массив уникальных пар значений дат и групп теста
datesGroups = orders[['date','group']].drop_duplicates()

# получаем агрегированные кумулятивные по дням данные о заказах 
ordersAggregated = datesGroups.apply(lambda x: orders[np.logical_and(orders['date'] <= x['date'], orders['group'] == x['group'])].agg({'date' : 'max', 'group' : 'max', 'transactionId' : 'nunique', 'visitorId' : 'nunique', 'revenue' : 'sum'}), axis=1).sort_values(by=['date','group'])

# получаем агрегированные кумулятивные по дням данные о посетителях интернет-магазина 
visitorsAggregated = datesGroups.apply(lambda x: visitors[np.logical_and(visitors['date'] <= x['date'], visitors['group'] == x['group'])].agg({'date' : 'max', 'group' : 'max', 'visitors' : 'sum'}), axis=1).sort_values(by=['date','group'])

# объединяем кумулятивные данные в одной таблице и присваиваем ее столбцам понятные названия
cumulativeData = ordersAggregated.merge(visitorsAggregated, left_on=['date', 'group'], right_on=['date', 'group'])
cumulativeData.columns = ['date', 'group', 'orders', 'buyers', 'revenue', 'visitors']

print(cumulativeData.head(5))